<a href="https://colab.research.google.com/github/SpandanaKalakonda/linkedinPosts/blob/main/few_shot_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [219]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import (
    load_img,
    img_to_array)
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [220]:
def create_siamese_network(input_shape):
    # Define the input layer
    input_layer = layers.Input(shape=input_shape)

    # Shared subnetwork (branch)
    shared_branch = models.Sequential([
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.Dense(128, activation='relu'),
        layers.Dense(64, activation='sigmoid'),
        layers.Dense(32, activation='softmax')
    ])

    # Apply the shared subnetwork to both inputs
    output_a = shared_branch(input_layer)
    output_b = shared_branch(input_layer)

    concatenated_output = layers.Concatenate()([output_a, output_b])

    model = models.Model(inputs=input_layer, outputs=concatenated_output)
    return model


In [221]:
#Siamese pair
def create_siamese_pair(num_classes, input_shape=(28, 28)):
    # Create Siamese network
    base_network = create_siamese_network(input_shape)

    # Define two input layers for pairs
    input_a = layers.Input(shape=input_shape)
    input_b = layers.Input(shape=input_shape)

    # Apply Siamese network to both inputs
    output_a = base_network(input_a)
    output_b = base_network(input_b)

    # Calculate L1 distance
    l1_distance = layers.Lambda(lambda x: tf.keras.backend.abs(x[0] - x[1]))([output_a, output_b])

    # Use a dense layer with sigmoid activation
    classification_output = layers.Dense(1, activation='sigmoid')(l1_distance)

    # Create Siamese pair model
    siamese_pair_model = models.Model(inputs=[input_a, input_b], outputs=classification_output)
    optimizer = optimizers.Adam(learning_rate=0.01)
    # Compile the model
    siamese_pair_model.compile(optimizer=optimizer, loss='contrastive_loss', metrics=['accuracy'])

    return siamese_pair_model


In [222]:
def preprocess_image(img):
    img = img.resize((28, 28), resample=Image.LANCZOS).convert('L')  # Resize and convert to grayscale
    img_array = np.array(img) / 255.0
    return img_array

In [223]:
image_gucci1 = Image.open('/content/gucci_logo1.jpeg')
image_gucci2 = Image.open('/content/gucci_logo2.jpeg')
image_doremon1 = Image.open('/content/doremon_1.png')
image_doremon2 = Image.open('/content/doremon_2.png')
image_nobita = Image.open("/content/nobita.jpeg")
image_nobita2 = Image.open("/content/nobita2.jpeg")
image_bat1 = Image.open("/content/circket_bat1.jpeg")
image_bat2 = Image.open("/content/circket_bat2.jpeg")

In [224]:
gucci_logo_1 = preprocess_image(image_gucci1)
gucci_logo_2 = preprocess_image(image_gucci2)
doremon_1 = preprocess_image(image_doremon1)
doremon_2 = preprocess_image(image_doremon2)
nobita = preprocess_image(image_nobita)
nobita2 = preprocess_image(image_nobita2)
bat1 = preprocess_image(image_bat1)
bat2 = preprocess_image(image_bat2)

In [225]:
#create pairs
pairs = [
    (gucci_logo_1, gucci_logo_2),
    (doremon_1, doremon_2),
    (nobita, nobita2),
    (bat1, bat2)
     # Add more pairs as needed
]

In [226]:
num_classes = 4
siamese_pair_model = create_siamese_pair(num_classes)

In [227]:
# Create a pair for prediction
pair_for_prediction = np.array([[doremon_1,doremon_2]])

# Make predictions using the trained Siamese model
predictions = siamese_pair_model.predict([pair_for_prediction[:, 0], pair_for_prediction[:, 1]])

# Print the predicted similarity score
print("Similarity Score:", predictions[0][0])


1/1 [==============================] - 0s 213ms/step
Similarity Score: 0.50008047


In [228]:
# Create a pair for prediction
pair_for_prediction = np.array([[gucci_logo_1,gucci_logo_2]])

# Make predictions using the trained Siamese model
predictions = siamese_pair_model.predict([pair_for_prediction[:, 0], pair_for_prediction[:, 1]])

# Print the predicted similarity score
print("Similarity Score:", predictions[0][0])


1/1 [==============================] - 0s 29ms/step
Similarity Score: 0.5001001


In [229]:
# Create a pair for prediction
pair_for_prediction = np.array([[nobita,nobita2]])

# Make predictions using the trained Siamese model
predictions = siamese_pair_model.predict([pair_for_prediction[:, 0], pair_for_prediction[:, 1]])

# Print the predicted similarity score
print("Similarity Score:", predictions[0][0])


1/1 [==============================] - 0s 25ms/step
Similarity Score: 0.49990895


In [230]:
# Create a pair for prediction
pair_for_prediction = np.array([[nobita, doremon_2]])

# Make predictions using the trained Siamese model
predictions = siamese_pair_model.predict([pair_for_prediction[:, 0], pair_for_prediction[:, 1]])

# Print the predicted similarity score
print("Similarity Score:", predictions[0][0])

1/1 [==============================] - 0s 26ms/step
Similarity Score: 0.500204
